In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler ,OneHotEncoder # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.neighbors import KNeighborsTransformer
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from category_encoders import TargetEncoder

In [2]:
df=pd.read_csv('data/car.csv')

In [3]:
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [4]:
df=df.drop_duplicates(keep='first')

In [59]:
X=df.drop(columns='selling_price',axis=1)
y=df['selling_price']

In [6]:
X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [64]:
numerical_features=X.select_dtypes(exclude='object').columns
categorical_features=X.drop(columns='name').select_dtypes(include='object').columns
categorical_features

Index(['fuel', 'seller_type', 'transmission', 'owner'], dtype='object')

In [79]:
fuel_c=['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric']
seller_type_c=['Individual', 'Dealer', 'Trustmark Dealer']
transmission_c=['Manual', 'Automatic']
owner_c=['First Owner', 'Second Owner', 'Fourth & Above Owner','Third Owner', 'Test Drive Car']
name_C=X.name.unique()



array(['Maruti 800 AC', 'Maruti Wagon R LXI Minor',
       'Hyundai Verna 1.6 SX', ..., 'Mahindra Verito 1.5 D6 BSIII',
       'Toyota Innova 2.5 VX (Diesel) 8 Seater BS IV',
       'Hyundai i20 Magna 1.4 CRDi'], dtype=object)

In [81]:
## Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[fuel_c,seller_type_c,transmission_c,owner_c])),
    ('TargetEncoder',TargetEncoder(categories=[name_C]))
    ('scaler',StandardScaler())
    ]

)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorical_cols)
])

<>:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_535/2565372771.py:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('TargetEncoder',TargetEncoder(categories=[name_C]))
/tmp/ipykernel_535/2565372771.py:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('TargetEncoder',TargetEncoder(categories=[name_C]))
/tmp/ipykernel_535/2565372771.py:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('TargetEncoder',TargetEncoder(categories=[name_C]))
/tmp/ipykernel_535/2565372771.py:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('TargetEncoder',TargetEncoder(categories=[name_C]))
/tmp/ipykernel_535/2565372771.py:16: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('TargetEncoder',TargetEncoder(categories=[name_C]))
/

TypeError: 'TargetEncoder' object is not callable

In [82]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [60]:
'''from category_encoders import TargetEncoder

cat1=X.columns[0]
#categorical_features=X.select_dtypes(include='object').columns
numeric_transformer=StandardScaler()
oh_transformaer=OneHotEncoder()
TargetEncoder=TargetEncoder()
preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',OneHotEncoder,categorical_features),
        ('StandardScaler',StandardScaler,numerical_features),
        ('TargetEncoder',TargetEncoder,'name')
    ]        
)
'''

In [83]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

TypeError: Cannot clone object. You should provide an instance of scikit-learn estimator instead of a class.

In [62]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,testsize=0.2,random_state=32)
X_train.shape

TypeError: got an unexpected keyword argument 'testsize'

In [ ]:
regression=LinearRegression()
regression.fit(X_train,y_train)

In [ ]:
regression.coef_

In [ ]:
regression.intercept_

In [ ]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

In [ ]:

model_list